###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2023 Semester 1

## Assignment 2: Book Rating Prediction

**Student IDs:**     1264462 & 1269549

# README

* To run the complete code in Jupyter Notebook, simply click Kernel > Restart & Run All.
    * Note that it might take a while to generate the output - the slowest model takes about < 7 minutes.
    * Alternatively, select the relevant cell, and click 'Run'.
* Code lines commented out with `##` are for printing out information that may be of interest. You may uncomment it to see the output.
* The dataset files (both the original and preprocessed) are assumed to exist in a folder named '**Datasets**', which resides in the same directory as this notebook.
* There are two helper notebooks in the same directory, namely "TextPreprocessing.ipynb" and "HelperFunctions.ipynb". Here we make use of the IPython built-in magic command (`%run`) to run these files. You do not need to open and run them separately.
* This code makes use of a library "mlxtend". To install it, run in the terminal: `pip install mlxtend`.
    * More details about this library can be found here: https://rasbt.github.io/mlxtend/.
* Each model may use a different version of the preprocessed dataset. Running this code generates the "train_df_ohe.csv" and "train_df_oe.csv" file. 
    * We have generated the other versions beforehand ("train_df_ohe_300", "train_df_oe_300"), which only requires a small change in a function parameter.
    * All required, preprocessed datasets lie in the 'Datasets' folder.
* This code also generates a file of the format "[model]_pred.csv", which contains the final predictions made by the selected model on the test set.

In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.feature_selection import chi2, SelectKBest, SelectFromModel

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%run TextPreprocessing.ipynb
%run HelperFunctions.ipynb

In [4]:
DATASET_DIR = "./Datasets/"
CLASS_LABEL = "rating_label"

In [5]:
# Load datasets
train_df = pd.read_csv(DATASET_DIR + "book_rating_train.csv", index_col = False, delimiter = ',', header=0)
test_df = pd.read_csv(DATASET_DIR + "book_rating_test.csv", index_col = False, delimiter = ',', header=0)
entire_df = pd.concat([train_df, test_df])

# 1. Data Preprocessing

Note: Here we show the individual preprocessing steps for the training set for clarity. At the end we pull everything together to preprocess the test set.

## Preprocess String features: Authors and Publisher

In [6]:
CATEGORICAL_FEATURES = ['Authors', 'Publisher']
MISSING_CAT_VAL = ''

In [7]:
train_df = train_df.drop('Language', axis=1)  # this feature has too many missing values

for df in [train_df, entire_df]:
    for feature in CATEGORICAL_FEATURES:
        df[feature] = df[feature].fillna(MISSING_CAT_VAL)  # impute missing values
        df[feature] = df[feature].apply(lambda x: preprocess(x, stop_words_removal=False, lemmatize=False, min_word_len=0))

### One-Hot Encoding

In [8]:
from sklearn.preprocessing import OneHotEncoder

OHE = OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=3, sparse=False)
OHE.fit(entire_df[CATEGORICAL_FEATURES])

OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=3,
              sparse=False, sparse_output=False)

In [9]:
def ohe_transform(df, has_labels=False):
    """
    Transforms the dataset by one-hot-encoding, on the categorical features only.
    `has_label` indicates whether the DataFrame contains the class labels.
        If True, these are moved to the last column.
    Returns the transformed DataFrame.
    """
    transformed_mat = OHE.transform(df[CATEGORICAL_FEATURES])
    transformed_cat_df = pd.DataFrame(transformed_mat).set_axis(OHE.get_feature_names_out(), axis=1, inplace=False)
    transformed_df = pd.concat([df.reset_index(drop=True), transformed_cat_df], axis=1)
    transformed_df = transformed_df.drop(CATEGORICAL_FEATURES, axis=1)  # drop the original attributes
    
    if has_labels:
        # move rating_label to the last column
        labels = transformed_df[CLASS_LABEL]
        transformed_df = pd.concat([transformed_df.drop([CLASS_LABEL], axis=1), labels], axis=1)
    
    return transformed_df

In [10]:
ALPHA = 0.05

# Select one-hot-encoded features using chi2
x2 = SelectKBest(chi2, k='all')
x2.fit(ohe_transform(train_df)[OHE.get_feature_names_out()], train_df[CLASS_LABEL])
pvals = pd.DataFrame(x2.pvalues_, index=x2.feature_names_in_, columns=['p-value'])
## print(pvals)

INSIG_OHE_FEATURES = pvals[pvals['p-value'] >= ALPHA].index.tolist()  # insignificant encoded features
sig_ohe_features = pvals[pvals['p-value'] < ALPHA].index.tolist()
print(len(sig_ohe_features), "features are significant.")
## sig_ohe_features

497 features are significant.


In [11]:
train_df = ohe_transform(train_df, has_labels=True)
train_df = train_df.drop(INSIG_OHE_FEATURES, axis=1)

## Discretize 'Numerical' features

In [12]:
from sklearn.preprocessing import KBinsDiscretizer

def discretize(df, feature, discretizer=None, strategy='kmeans', n_bins=5):
    """
    Discretizes the feature in the given DataFrame.
    - discretizer: the discretizer; 
      If None, creates a KBinsDiscretizer for ordinal data, with the specified strategy and number of bins.
    - strategy: the discretization strategy (one of ['kmeans', 'quantile', 'uniform']).
    - n_bins: the number of bins.
    Returns the transformed dataset and discretizer used.
    """
    
    if discretizer is None:
        discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy=strategy)
        discretizer.fit(df[[feature]])
        
    transformed_df = df.copy()
    transformed_df[feature] = discretizer.transform(df[[feature]])
    
    return transformed_df, discretizer

In [13]:
# PublishYear
train_df, DISCRETIZER_PY = discretize(train_df, 'PublishYear', strategy='uniform', n_bins=15)

# PublishMonth
train_df, DISCRETIZER_PM = discretize(train_df, 'PublishMonth', strategy='quantile', n_bins=6)

# PublishDay
train_df, DISCRETIZER_PD = discretize(train_df, 'PublishDay', strategy='quantile', n_bins=11)

# pagesNumber
train_df, DISCRETIZER_PN = discretize(train_df, 'pagesNumber', strategy='kmeans', n_bins=7)

## Text Pre-processing

In [14]:
TEXT_FEATURES = ['Name', 'Description']

In [15]:
# set max_features=300 to generate the dataset version for the Logistic Regression classifier
train_df_name, VECTORIZER_NAME = preprocess_text_feature(train_df, 'Name', ngram=2, delimiter='_')  # distinguish from Description features
train_df_desc, VECTORIZER_DESC = preprocess_text_feature(train_df, 'Description', ngram=2)

In [16]:
# Bind them together
train_df = pd.concat([train_df.reset_index(drop=True), train_df_name, train_df_desc], axis=1)
train_df = train_df.drop(TEXT_FEATURES, axis=1)  # drop the original columns

In [17]:
# Move rating_label to the last column
labels = train_df[CLASS_LABEL].astype('category')
train_df = pd.concat([train_df.drop([CLASS_LABEL], axis=1), labels], axis=1) 

## Normalization

In [18]:
from sklearn.preprocessing import MinMaxScaler

train_df_X = train_df.iloc[:,:-1]
train_df_y = train_df.iloc[:,-1]
SCALER = MinMaxScaler()
scaled = SCALER.fit_transform(train_df_X)

In [19]:
scaled_X = pd.DataFrame(scaled, columns=train_df_X.columns)
train_df = pd.concat([scaled_X, train_df_y], axis=1)
train_df

,PublishYear,PublishMonth,PublishDay,pagesNumber,Authors_,Authors_aaron mccollough,Authors_abu hamid al ghazali,Authors_agatha christie,Authors_alba de c spedes,Authors_alexander kent,...,book offer,young woman,science fiction,well known,br em,real life,tell story,african american,san francisco,rating_label
0,0.928571,0.6,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1,0.857143,1.0,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.928571,0.2,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,0.928571,0.8,0.000000,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
4,0.928571,0.6,0.166667,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23058,0.857143,0.6,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
23059,0.928571,0.6,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
23060,0.857143,0.2,0.500000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
23061,0.928571,0.4,0.666667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [26]:
# Save the preprocessed training set
train_df.to_csv(DATASET_DIR + "train_df.csv", index=False)

## Altogether

In [20]:
def preprocess_test_df(df):
    """
    Preprocesses the test dataset and returns the preprocessed version as a DataFrame.
    """
    df = df.copy()
    
    # preprocess string features
    for feature in CATEGORICAL_FEATURES:
        df[feature] = df[feature].fillna(MISSING_CAT_VAL)  # imputation
        df[feature] = df[feature].apply(lambda x: preprocess(str(x), stop_words_removal=False, lemmatize=False, min_word_len=0))
    df = ohe_transform(df).drop(INSIG_OHE_FEATURES, axis=1)  # one-hot-encoding
    
    # preprocess numerical features
    df = discretize(df, 'PublishDay', discretizer=DISCRETIZER_PD)[0]
    df = discretize(df, 'PublishMonth', discretizer=DISCRETIZER_PM)[0]
    df = discretize(df, 'PublishYear', discretizer=DISCRETIZER_PY)[0]
    df = discretize(df, 'pagesNumber', discretizer=DISCRETIZER_PN)[0]
    
    # preprocess text features
    df_name = preprocess_text_feature(df, 'Name', vectorizer=VECTORIZER_NAME, delimiter='_')[0]
    df_desc = preprocess_text_feature(df, 'Description', vectorizer=VECTORIZER_DESC)[0]
    
    # discard the obsolete original features and unwanted features
    df = pd.concat([df.reset_index(drop=True), df_name, df_desc], axis=1)
    df = df.drop(['Name', 'Description', 'Language'], axis=1)
    
    # normalize
    df = pd.DataFrame(SCALER.transform(df), columns=df.columns)
        
    return df

In [22]:
# Apply to the test set
test_df = preprocess_test_df(test_df)
test_df.to_csv(DATASET_DIR + "test_df.csv", index=False)

# 2. Building the models

Training sets for each model:
- `logr`: train_df_ohe_300.csv, the one-hot-encoded dataset with 300 'Name' and 'Description' features, respectively.
        This is generated by setting `max_features=300` in the `preprocess_text_feature()` function.
- `rf`, `cvsnb`, baselines: train_df_ohe.csv, the one-hot-encoded dataset with 50 'Name' and 'Description' features, respectively.
- `sclf`: train_df_oe.csv, the ordinal-encoded dataset with 50 'Name' and 'Description' features, respectively.

In [25]:
## train_df_ohe = pd.read_csv(DATASET_DIR + "train_df_ohe.csv", keep_default_na=False)  # Alternative
X_train = train_df.iloc[:,:-1]
y_train = train_df.iloc[:,-1]

## 1) Random Forest

## 2) StackingClassifier

In [24]:
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.classifier import StackingCVClassifier

In [ ]:
# Gaussian NB
gnb = GaussianNB()
# Linear SVM
linearSVM = svm.LinearSVC(random_state=30027, C=1)
# Logistic Regression
logr = LogisticRegression(solver='sag', max_iter=100)
# Decision tree
dt = DecisionTreeClassifier(max_depth = 400, random_state = 30027)
# 3NN
knn = KNeighborsClassifier(n_neighbors=3)

# Stack
sclf = StackingCVClassifier(classifiers=[gnb, linearSVM, logr, dt, knn], 
                            meta_classifier=LogisticRegression(),
                            cv=2,
                            random_state=30027)

sclf.fit(X_train, y_train)

In [ ]:
# This is slow - be patient!
y_pred_sclf, sclf_report = cross_val_report(sclf, X_train, y_train)
sclf_report

## 3) Logistic Regression

In [ ]:
train_df_logr = pd.read_csv(DATASET_DIR + "train_df_ohe_300.csv", keep_default_na=False)
X_train_logr = train_df_logr.iloc[:,:-1]

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logr = LogisticRegression(solver='sag', max_iter=500)  # uses Stochastic Average Gradient descent solver
logr.fit(X_train_logr, y_train)

In [ ]:
selector = SelectFromModel(logr, prefit=True).fit(X_train_logr, y_train)
LR_FEATURES = [X_train_logr.columns[i] for i in selector.get_support(indices=True)]

In [ ]:
logr.fit(X_train_logr[LR_FEATURES], y_train)
y_pred_logr, logr_report = cross_val_report(logr, X_train_logr[LR_FEATURES], y_train)
logr_report

## 4) Stacking Naive Bayes

In [ ]:
from mlxtend.classifier import StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB, CategoricalNB

In [ ]:
GAUS_FEATURES = ['pagesNumber', 'PublishYear']
CAT_FEATURES = ['Authors']
MN_FEATURES = chi2_select_features(X_train_oe[X_train_oe.columns[6:]], y_train_oe)  # only filtering the text features
N_AUTHORS_VALS = len(ORD_ENCODER.categories_[0])  # number of unique Authors categories

X_train_oe = X_train_oe[GAUS_FEATURES + CAT_FEATURES + MN_FEATURES]
## X_train_oe

In [ ]:
gnb = make_pipeline(ColumnSelector(GAUS_FEATURES),
                      GaussianNB())
cnb = make_pipeline(ColumnSelector(CAT_FEATURES),
                      CategoricalNB(alpha=0.11, min_categories=N_AUTHORS_VALS))
mnb = make_pipeline(ColumnSelector(MN_FEATURES),
                      MultinomialNB(alpha=0.26))

cvsnb = StackingCVClassifier(classifiers=[gnb, cnb, mnb], 
                            meta_classifier=CategoricalNB(),
                            random_state=30027)

cvsnb.fit(X_train_oe, y_train_oe)

In [ ]:
y_pred_cvsnb, cvsnb_report = cross_val_report(cvsnb, X_train_oe, y_train_oe)
cvsnb_report

## 5) Baselines

### 0R

In [ ]:
from sklearn.dummy import DummyClassifier

zero_r = DummyClassifier(strategy='most_frequent')
y_pred_zero_r, zero_r_report = cross_val_report(zero_r, X_train, y_train)
zero_r_report

### 1R

In [ ]:
from sklearn.tree import DecisionTreeClassifier

one_r = DecisionTreeClassifier(max_depth=1)
y_pred_one_r, one_r_report = cross_val_report(one_r, X_train, y_train)
one_r_report

# 3. Final predictions on the test set

In [ ]:
# Stacking Clf
X_test_sclf = pd.read_csv(DATASET_DIR + "test_df_ohe.csv", keep_default_na=False).drop(INSIG_OHE_FEATURES, axis=1)
get_pred_df(sclf.predict(X_test_sclf)).to_csv("sclf_pred.csv", index=False)
pd.read_csv("sclf_pred.csv")  # how the results look

In [ ]:
# Random Forest
X_test_rf = pd.read_csv(DATASET_DIR + "test_df_ohe.csv", keep_default_na=False)
get_pred_df(rf.predict(X_test_rf[RF_FEATURES])).to_csv("rf_pred.csv", index=False)

In [ ]:
# Logistic Regression
X_test_logr = pd.read_csv(DATASET_DIR + "test_df_ohe_300.csv", keep_default_na=False)
get_pred_df(logr.predict(X_test_logr[LR_FEATURES])).to_csv("logr_pred.csv", index=False)

In [ ]:
# Stacking NB
X_test_cvsnb = pd.read_csv(DATASET_DIR + "test_df_oe.csv", keep_default_na=False)
get_pred_df(cvsnb.predict(X_test_cvsnb)).to_csv("cvsnb_pred.csv", index=False)